# Introduction to PDDL

In the first example, we looked at manually defining a planning problem.

However, there exist entire modeling languages to more easily define these.
The most common one is Planning Domain Description Language, or PDDL.

Unified Planning offers a PDDL reader that you can use to directly read these files.

In [41]:
from unified_planning.io import PDDLReader
from unified_planning.shortcuts import *

reader = PDDLReader()
pddl_problem = reader.parse_problem(
    "pddl/pick_place_domain.pddl",
    "pddl/pick_place_problem_01.pddl",
)
print(pddl_problem)

problem name = pick_and_place_problem_01

types = [bin, part]

fluents = [
  bool robot_at[b=bin]
  bool part_at[p=part, b=bin]
  bool holding[p=part]
  bool gripper_empty
  bool occupied[b=bin]
]

actions = [
  action move(bin b_from, bin b_to) {
    preconditions = [
      robot_at(b_from)
    ]
    effects = [
      robot_at(b_from) := false
      robot_at(b_to) := true
    ]
  }
  action pick(part p, bin b) {
    preconditions = [
      (robot_at(b) and part_at(p, b) and gripper_empty)
    ]
    effects = [
      part_at(p, b) := false
      holding(p) := true
      gripper_empty := false
      occupied(b) := false
    ]
  }
  action place(part p, bin b) {
    preconditions = [
      (robot_at(b) and holding(p) and (not occupied(b)))
    ]
    effects = [
      holding(p) := false
      gripper_empty := true
      part_at(p, b) := true
      occupied(b) := true
    ]
  }
]

objects = [
  bin: [bin_1, bin_2, bin_3]
  part: [part_1, part_2]
]

initial fluents default = [
  bool robot

Once you have read in domain and problem files from PDDL, you can perform planning in the same way as manually defined domains!

In [42]:
with OneshotPlanner(name="tamer") as planner:
    result = planner.solve(pddl_problem)
    if result.plan is None:
        print("Found no plan :(")
    else:
        print(result.plan)

  *** Credits ***
  * In operation mode `OneshotPlanner` at line 1 of `/tmp/ipykernel_475236/2296574505.py`, you are using the following planning engine:
  * Engine name: Tamer
  * Developers:  FBK Tamer Development Team
  * Description: Tamer offers the capability to generate a plan for classical, numerical and temporal problems.
  *              For those kind of problems tamer also offers the possibility of validating a submitted plan.

SequentialPlan:
    move(bin_3, bin_1)
    pick(part_1, bin_1)
    move(bin_1, bin_3)
    place(part_1, bin_3)
    move(bin_3, bin_2)
    pick(part_2, bin_2)
    move(bin_2, bin_1)
    place(part_2, bin_1)
    move(bin_1, bin_2)


By defining your domains and problems in separate PDDL files, the amount of code written here is quite minimal!

The idea is that you can reuse the same domain file along with any arbitrary number of problem files.

Feel free to make your own additional problem files in the `pddl` subfolder, and see how things go!

Another thing you can do once you have a Unified Planning problem loaded is **simulation**.
In other words, you can start from a specific initial state and propagate the effects of actions to simulate how the state changes.
This is useful for many reasons, including:

* Tracking the symbolic state in a world model as an agent executes actions in the real world -- this helps have an up-to-date initial state for replanning after a plan is completed or fails.
* Validating whether an action (or set of actions) is still applicable in the event that the state changed during execution.

Unified Planning offers a `SequentialSimulator` class to do this, which we will explore now.

In [43]:
simulator = SequentialSimulator(pddl_problem)
initial_state = simulator.get_initial_state()
print(initial_state)

{robot_at(bin_3): true, gripper_empty: true, part_at(part_1, bin_1): true, part_at(part_2, bin_2): true, occupied(bin_1): true, occupied(bin_2): true, robot_at(bin_1): false, robot_at(bin_2): false, part_at(part_2, bin_1): false, part_at(part_1, bin_2): false, part_at(part_1, bin_3): false, part_at(part_2, bin_3): false, holding(part_1): false, holding(part_2): false, occupied(bin_3): false}


To forward propagate a state given some actions, you can simulate as follows:

In [44]:
from unified_planning.plans import ActionInstance

move = pddl_problem.action("move")
bin_1 = pddl_problem.object("bin_1")
bin_2 = pddl_problem.object("bin_2")
bin_3 = pddl_problem.object("bin_3")

new_state = simulator.apply(initial_state, ActionInstance(move, (bin_3, bin_2)))
print(new_state)

{robot_at(bin_3): false, robot_at(bin_2): true, gripper_empty: true, part_at(part_1, bin_1): true, part_at(part_2, bin_2): true, occupied(bin_1): true, occupied(bin_2): true, robot_at(bin_1): false, part_at(part_2, bin_1): false, part_at(part_1, bin_2): false, part_at(part_1, bin_3): false, part_at(part_2, bin_3): false, holding(part_1): false, holding(part_2): false, occupied(bin_3): false}


Notice that in the above case, we already knew that the action `move(bin_3, bin_2)` was valid.
What if we try an invalid action?

In [45]:
new_state = simulator.apply(initial_state, ActionInstance(move, (bin_1, bin_2)))  # robot is at bin_3, though!
print(new_state)  # this is None!

None


Instead, you can check if actions are applicable with `SequentialSimulator`.

In [46]:
move_bin_1_bin_2 = ActionInstance(move, (bin_1, bin_2))

is_applicable = simulator.is_applicable(initial_state, move_bin_1_bin_2)  # robot is at bin_3, though!
print(f"Is action applicable? {is_applicable}")

unsatisfied_conditions, reasons = simulator.get_unsatisfied_conditions(initial_state, move_bin_1_bin_2)
print(f"Unsatisfied conditions:\n{unsatisfied_conditions}")

Is action applicable? False
Unsatisfied conditions:
[robot_at(bin_1)]


With this information, you can safely propagate states using an entire sequence of actions.
This, of course, can come from a task plan!

In [47]:

pick = pddl_problem.action("pick")
place = pddl_problem.action("place")
part_1 = pddl_problem.object("part_1")
part_2 = pddl_problem.object("part_2")

task_plan = [
    ActionInstance(move, (bin_3, bin_2)),
    ActionInstance(pick, (part_2, bin_2)),
    ActionInstance(move, (bin_2, bin_1)),
    ActionInstance(place, (part_2, bin_1)),
]

cur_state = initial_state
for action in task_plan:
    if not simulator.is_applicable(cur_state, action):
        unsatisfied_conditions, _ = simulator.get_unsatisfied_conditions(cur_state, action)
        print(f"Cannot apply action: {action}\nUnsatisfied conditions: {unsatisfied_conditions}")
        break

    cur_state = simulator.apply(cur_state, action)
    print(f"Applied action: {action}")

print(f"\nFinal simulated state: {cur_state}")
    
        

Applied action: move(bin_3, bin_2)
Applied action: pick(part_2, bin_2)
Applied action: move(bin_2, bin_1)
Cannot apply action: place(part_2, bin_1)
Unsatisfied conditions: [(not occupied(bin_1))]

Final simulated state: {robot_at(bin_2): false, robot_at(bin_1): true, part_at(part_2, bin_2): false, holding(part_2): true, gripper_empty: false, occupied(bin_2): false, robot_at(bin_3): false, part_at(part_1, bin_1): true, occupied(bin_1): true, part_at(part_2, bin_1): false, part_at(part_1, bin_2): false, part_at(part_1, bin_3): false, part_at(part_2, bin_3): false, holding(part_1): false, occupied(bin_3): false}


Notice that with the simulator, we could determine this was an invalid task plan!

In this case, the robot was unable to place `part_2` in `bin_1`, because the bin was already occupied by `part_1`.

**Fun fact:** When I was putting this example together, at first this action executed correctly even though I expected it to fail!
Indeed, this uncovered a bug in my domain description, since there were no fluents that prevented multiple objects from being placed in the same bin.
So, by using `SequentialSimulator` and validating plans, I could identify the issue and add the new `occupied(?b - bin)` fluent.